In [3]:
import ee
import geemap

In [10]:
import os

In [4]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
site = ee.FeatureCollection('users/Introduction/study_area')
Map.addLayer(site, {}, "study_area")

In [16]:

# Load MODIS vegetation indices data and subset a decade of images.
vegIndices = ee.ImageCollection('MODIS/006/MOD13A1') \
                     .filter(ee.Filter.date('2010-01-01', '2020-01-01')) \
                     .select('NDVI')

def func_xmt(image):
  return image.multiply(0.0001) \
    .copyProperties(image, ["system:time_start"])

ndvi = vegIndices.map(func_xmt).mean().clip(site)

Map.addLayer(ndvi, {'palette': ["#640000",  "#ff0000",  "#ffff00",  "#00c800",  "#006400"], 'min': 0.1, 'max': 0.8}, "NDVI")


In [17]:
# 3. Classify NDVI into 5 classes
ndvi2 = ee.Image(1) \
          .where(ndvi.gt(0.0).And(ndvi.lte(0.2)), 2) \
          .where(ndvi.gt(0.2).And(ndvi.lte(0.4)), 3) \
          .where(ndvi.gt(0.4).And(ndvi.lte(0.6)), 4) \
          .where(ndvi.gt(0.6), 5)

ndvi2 = ndvi2.clip(site)
# Add map layers
Map.addLayer(ndvi2, {'min': 1, 'max': 5, 'palette': ["#ff0000",'#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Classified NDVI',True)


In [18]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'ndvi2.tif')

In [19]:
geemap.ee_export_image(ndvi2, filename=filename, scale=500, region=site.geometry(), file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Mukesh\Downloads\ndvi2.tif


In [20]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [29]:
# Import image collection
modis = ee.ImageCollection('MODIS/006/MOD11A2')
print(modis)

# A start date is defined and the end date is determined by advancing 1 year from the start date.
start = ee.Date('2015-01-01')
end = ee.Date('2020-12-31')
dateRange = ee.DateRange(start, end)

# Filter the LST collection to include only images from time frame and select day time temperature band

modLSTday = modis.filterDate(dateRange).select('LST_Day_1km')

# Scale to Kelvin and convert to Celsius, set image acquisition time.

def func_dmp(image):
  return image.multiply(0.02) \
              .subtract(273.5) \
              .copyProperties(image, ["system:time_start"])

modC = modLSTday.map(func_dmp)


#Clip to roi
LSTclip = modC.mean().clip(site)

# Add clipped image layer to the map.

# 3. Classify NDVI into 5 classes
temp = ee.Image(0) \
          .where(LSTclip.gt(0.0).And(LSTclip.lte(15)), 1) \
          .where(LSTclip.gt(15).And(LSTclip.lte(20)), 2) \
          .where(LSTclip.gt(20).And(LSTclip.lte(25)), 3) \
          .where(LSTclip.gt(25).And(LSTclip.lte(30)), 4) \
          .where(LSTclip.gt(30).And(LSTclip.lte(35)), 5) \
          .where(LSTclip.gt(35).And(LSTclip.lte(40)), 6) \
          .where(LSTclip.gt(40), 7)

# Add map layers
Map.addLayer(temp,  {'min': 1, 'max': 8, 'palette': ['#ffffcc','#ffeda0','#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c']},  'Classified LST',True)


ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "MODIS/006/MOD11A2"
      }
    }
  }
})


In [30]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'LST.tif')

In [32]:
geemap.ee_export_image(temp, filename=filename, scale=500, region=site.geometry(), file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Mukesh\Downloads\LST.tif
